# Python SQL
## - CRUD Operation

In [1]:
import json
import pymysql

In [2]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [3]:
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port'],
)

## ★ 테이블 생성

In [4]:
sql_user = """
    CREATE TABLE if NOT EXISTS users (
        uid VARCHAR(20) NOT NULL PRIMARY KEY,
        pwd CHAR(44),
        uname VARCHAR(20) NOT NULL,
        reg_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        is_deleted INT DEFAULT 0
    );
"""

In [5]:
cur = conn.cursor()
cur.execute(sql_user)

0

## ★ 테이블 변경

In [6]:
cur.execute('alter table users add column email varchar(40);')

OperationalError: (1060, "Duplicate column name 'email'")

## ★ 데이터 입력

In [7]:
cur = conn.cursor()
cur.execute("INSERT INTO users(uid, uname) VALUES('admin','관리자');")

1

In [8]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 14, 2, 11), 0, None)
('diy', None, '대조영', datetime.datetime(2021, 6, 9, 11, 32, 20), 0, 'admin@korea.com')
('djy', None, '대조영', datetime.datetime(2021, 6, 9, 13, 49, 46), 0, None)
('eskim', None, '김은숙', datetime.datetime(2021, 6, 9, 11, 32, 20), 0, None)
('gdhong', None, '홍길동', datetime.datetime(2021, 6, 9, 11, 32, 20), 0, None)
('jbpark', None, '박재범', datetime.datetime(2021, 6, 9, 11, 32, 20), 0, None)
('vincenzo', None, '빈센조', datetime.datetime(2021, 6, 9, 11, 32, 20), 0, None)
('wjlee', None, '이우정', datetime.datetime(2021, 6, 9, 11, 32, 20), 0, None)


In [10]:
# 입력한 데이터를 즉시 flush 하려면 conn.commit() 추가
sql_insert_multi = """
    INSERT INTO users(uid, uname) 
        VALUES('eskim','김은숙'),('wjlee','이우정');
"""
cur.execute(sql_insert_multi)
conn.commit()

IntegrityError: (1062, "Duplicate entry 'eskim' for key 'users.PRIMARY'")

In [11]:
sql_select = """
    SELECT uid, uname, 
    DATE_FORMAT(reg_date, '%Y-%m-%d %h:%i') AS reg_date,
    is_deleted, email FROM users;
"""
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', '관리자', '2021-06-09 02:02', 0, None)
('diy', '대조영', '2021-06-09 11:32', 0, 'admin@korea.com')
('djy', '대조영', '2021-06-09 01:49', 0, None)
('eskim', '김은숙', '2021-06-09 11:32', 0, None)
('gdhong', '홍길동', '2021-06-09 11:32', 0, None)
('jbpark', '박재범', '2021-06-09 11:32', 0, None)
('vincenzo', '빈센조', '2021-06-09 11:32', 0, None)
('wjlee', '이우정', '2021-06-09 11:32', 0, None)


In [12]:
# 파라메터 이용, placeholder
uid = 'djy'
uname = '대조영'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
cur.execute(sql, (uid, uname))
conn.commit()

IntegrityError: (1062, "Duplicate entry 'djy' for key 'users.PRIMARY'")

In [13]:
uid = 'gdhong'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, '홍길동');"
cur.execute(sql, (uid, ))  # uid만 넣을 경우, 튜플이 아니기 때문에 에러
conn.commit()

IntegrityError: (1062, "Duplicate entry 'gdhong' for key 'users.PRIMARY'")

In [14]:
users = (('jbpark','박재범'),('vincenzo','빈센조'))
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
# 아래 for 문은 cur.executemany(sql, users) 로 대체 가능
for user in users:
    cur.execute(sql, user)
conn.commit()

IntegrityError: (1062, "Duplicate entry 'jbpark' for key 'users.PRIMARY'")

## ★ 데이터 삭제

In [15]:
cur = conn.cursor()
sql = "DELETE FROM users WHERE uid != 'admin';"
cur.execute(sql)
conn.commit()

In [16]:
import pandas as pd
uids = ['diy', 'eskim','gdhong','jbpark','vincenzo','wjlee']
unames = ['대조영','김은숙','홍길동','박재범','빈센조','이우정']
df = pd.DataFrame({'uid':uids,'uname':unames})
df

,uid,uname
0,diy,대조영
1,eskim,김은숙
2,gdhong,홍길동
3,jbpark,박재범
4,vincenzo,빈센조
5,wjlee,이우정


In [17]:
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
for i in df.index:
    cur.execute(sql, (df.uid[i], df.uname[i]))
conn.commit()

## ★ 데이터 수정

In [18]:
email = 'admin@korea.com'
uid = 'admin'
cur = conn.cursor()
sql = "UPDATE users SET email=%s WHERE uid=%s;"
cur.execute(sql, (email, uid))
conn.commit()

In [19]:
cur = conn.cursor()
sql = "UPDATE users SET email=%s WHERE uid=%s;"
for i in df.index:
    uid = df.uid[i]
    email = f'{uid}@email.com'
    cur.execute(sql, (email, uid))
conn.commit()

## ★ 데이터 조회

In [20]:
sql = """
    SELECT uid, uname, email,
    DATE_FORMAT(reg_date, '%Y-%m-%d %h:%i') AS reg_date
    FROM users WHERE is_deleted=0;
"""

In [21]:
# 한 레코드 조회
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchone()
result

('admin', '관리자', 'admin@korea.com', '2021-06-09 02:02')

In [22]:
# n개의 레코드 조회
cur.fetchmany(2)

(('diy', '대조영', 'diy@email.com', '2021-06-09 02:03'),
 ('eskim', '김은숙', 'eskim@email.com', '2021-06-09 02:03'))

In [23]:
# 모두 조회
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
for row in results:
    print(row)

('admin', '관리자', 'admin@korea.com', '2021-06-09 02:02')
('diy', '대조영', 'diy@email.com', '2021-06-09 02:03')
('eskim', '김은숙', 'eskim@email.com', '2021-06-09 02:03')
('gdhong', '홍길동', 'gdhong@email.com', '2021-06-09 02:03')
('jbpark', '박재범', 'jbpark@email.com', '2021-06-09 02:03')
('vincenzo', '빈센조', 'vincenzo@email.com', '2021-06-09 02:03')
('wjlee', '이우정', 'wjlee@email.com', '2021-06-09 02:03')


In [24]:
# 모두 조회한 뒤 데이터프레임으로 만들기
uids, unames, emails, regs = [],[],[],[]
for row in results:
    uids.append(row[0])
    unames.append(row[1])
    emails.append(row[2])
    regs.append(row[3])
df2 = pd.DataFrame({
    'uid': uids, 'uname': unames,
    'email': emails, 'reg_date': regs
})
df2

,uid,uname,email,reg_date
0,admin,관리자,admin@korea.com,2021-06-09 02:02
1,diy,대조영,diy@email.com,2021-06-09 02:03
2,eskim,김은숙,eskim@email.com,2021-06-09 02:03
3,gdhong,홍길동,gdhong@email.com,2021-06-09 02:03
4,jbpark,박재범,jbpark@email.com,2021-06-09 02:03
5,vincenzo,빈센조,vincenzo@email.com,2021-06-09 02:03
6,wjlee,이우정,wjlee@email.com,2021-06-09 02:03


## ★ 데이터 검색

In [25]:
keyword = '김'
sql = """
    SELECT uid FROM users 
    WHERE is_deleted=0 and uname like %s;
"""

In [26]:
# 결과가 하나라도 튜플로 데이터를 반환
cur = conn.cursor()
cur.execute(sql, keyword+'%')
result = cur.fetchone()
result

('eskim',)

In [27]:
result[0]

'eskim'

In [28]:
cur = conn.cursor()
cur.execute(sql, keyword+'%')
result = cur.fetchall()
result

(('eskim',),)

In [29]:
result[0][0]

'eskim'